# DLKit Overview

DLKit is a Python implementation of the OSIDs (www.osid.org).

Every service has a catalog object, which houses / contains additional objects. These catalogs help control authorization to the contained objects, and can be arranged hierarchically (with authorizations flowing down).

             MIT
            /   \
           /     \
       Physics   Math  

Someone with access to all of MIT could see both Math and Physics materials, but folks in Physics may not be able to see Math materials.

Let's start with a simple example and dig more in depth.

Note that all data generated with this tutorial is saved in JSON files to your harddrive, in a folder called `tutorial-data`, sibling to this `.ipynb` file. If you ever want to start over, just delete that folder and all its contents.

## Assessment Service

In the Assessment Service, the catalog is called a `bank`. `banks` contain other objects, called `item`, `assessment`, `assessmentOffered`, `assessmentTaken`, etc. We'll start with `item` and `assessment`.

To being with, you access each service through a `manager`. For example, an `AssessmentManager` gives you access to the various `bank` and `item` methods.

In order to get a `manager`, we go through the `runtime` -- in this tutorial, the `dlkit_edx` runtime (it's not very specific to edX, but that's what we call it). We will simulate a user and a test web request, to pass along. This username is automatically included with many types of data, such as when taking assessments.

In [3]:
from dlkit_edx import PROXY_SESSION, RUNTIME
from dlkit_edx.proxy_example import TestRequest

condition = PROXY_SESSION.get_proxy_condition()
dummy_request = TestRequest(username='tutorial_user@school.edu',
                            authenticated=True)
condition.set_http_request(dummy_request)
proxy = PROXY_SESSION.get_proxy(condition)
am = RUNTIME.get_service_manager('ASSESSMENT',
                                  proxy=proxy)

print am

**Banks**

Now that we have an `AssessmentManager`, we can see what `banks` exist in the system. Calling for "lists" of things returns a "thing list", like a `BankList` (a Python generator), and we can check the number of results with `.available()`.


In [4]:
print am.banks
print am.banks.available()

0


No banks!! Okay, let's create one. In `DLKit`, CRUD operations are done with `forms`. So let's get a `form` to create a new assessment `bank`, and assign it a `displayName` and `description` that we'll recognize later.

Notice that in the `get_bank_form_for_create()` method, we pass an empty list as an argument. This can be used to extend the base functionality of the `bank`, via record extensions. That is a more advanced feature we'll touch on later.

In [5]:
form = am.get_bank_form_for_create([])
form.display_name = "MyBank"
form.description = "For learning about DLKit"
bank = am.create_bank(form)
print bank
print bank.display_name.text
print bank.description.text
print str(bank.ident)

MyBank
For learning about DLKit
assessment.Bank%3A57fe54dccdfc5c3a0bc23b47%40ODL.MIT.EDU


You can see that our new `bank` has some attributes -- some that we assigned (`displayName` and `description`, and others that were created by `DLKit`, like `ident`). `displayName` and `description` are actually objects themselves, and can also contain language, format, and script data, which is why we call `display_name.text` and `description.text` above.

**Items**

Now that we have a `bank`, we can create assessment `items` in it. 

An `item` is what you might think of as a basic assessment question with the associated answers (right or wrong). There are many types of `item`s, including multiple choice, fill in the blank, short answer, etc.

Again, we can inspect to see if any exist.

In [7]:
print bank.items
print bank.items.available()

0


To create a new item, we'll grab a form. In the `OSIDS`, since `items` are not defined beyond a question and answer, we'll need to pass along a list of record extensions, so give the item some functionality. This requires some internal knowledge about `DLKit`, so for now well just use a simple multiple choice record plus accomodation for wrong-answers. 

In [12]:
from dlkit_edx.primitives import Type
from records.registry import ITEM_RECORD_TYPES
MULTIPLE_CHOICE_ITEM = Type(**ITEM_RECORD_TYPES['multi-choice'])
WRONG_ANSWER_ITEM = Type(**ITEM_RECORD_TYPES['wrong-answer'])

form = bank.get_item_form_for_create([MULTIPLE_CHOICE_ITEM, WRONG_ANSWER_ITEM])
form.display_name = "Basic addition question"
form.description = "addition question with fruit"
item = bank.create_item(form)

But where's the question?? We'll create that separately, with its own records. Recall:

        Item
          |--Question
          |--Answers
          
Since `questions` are distinct from `items`, students can be sent only the `question` object, without any danger of `answers` leaking. Note that the `question` `form` requires an extra `itemId` argument.

In [13]:
from records.registry import QUESTION_RECORD_TYPES
MULTIPLE_CHOICE_QUESTION = Type(**QUESTION_RECORD_TYPES['multi-choice-text'])

form = bank.get_question_form_for_create(item.ident, [MULTIPLE_CHOICE_QUESTION])
form.set_text("Which color do you prefer?")
form.add_choice("blue")
form.add_choice("red")
form.add_choice("yellow")
question = bank.create_question(form)

print question.get_choices()

[{'text': 'blue', 'id': '57fe61edcdfc5c3a0bc23b49', 'name': ''}, {'text': 'red', 'id': '57fe61edcdfc5c3a0bc23b4a', 'name': ''}, {'text': 'yellow', 'id': '57fe61edcdfc5c3a0bc23b4b', 'name': ''}]


The `item` also has a handle to its own `question`.

In [14]:
print item.get_question()

TypeError: 'NoneType' object has no attribute '__getitem__'

What?!?! Oh, wait ... because the `item` object we have was initialized before the `question` was actually created, we need to re-grab that `item` to get it in the "newest" state. Note that you can also get the same information by calling `is_current()` on the item.

In [29]:
print item.is_current()
item = bank.get_item(item.ident)
print item.get_question()
print item.get_question().get_choices()

False
[{u'text': u'blue', u'id': u'57fe61edcdfc5c3a0bc23b49', u'name': u''}, {u'text': u'red', u'id': u'57fe61edcdfc5c3a0bc23b4a', u'name': u''}, {u'text': u'yellow', u'id': u'57fe61edcdfc5c3a0bc23b4b', u'name': u''}]


We can also set up wrong / right `answers`, to be used in evaluating the "correctness" of the response. We indicate the type of `answer` with the `genusTypeId` property.

In [25]:
from records.registry import ANSWER_GENUS_TYPES, ANSWER_RECORD_TYPES
MULTIPLE_CHOICE_ANSWER = Type(**ANSWER_RECORD_TYPES['multi-choice'])
RIGHT_ANSWER = Type(**ANSWER_GENUS_TYPES['right-answer'])
WRONG_ANSWER = Type(**ANSWER_GENUS_TYPES['wrong-answer'])

form = bank.get_answer_form_for_create(item.ident, [MULTIPLE_CHOICE_ANSWER])
form.set_genus_type(RIGHT_ANSWER)
# We'll just set "blue" as the right answer, arbitrarily
form.add_choice_id('57fe61edcdfc5c3a0bc23b49')
answer1 = bank.create_answer(form)

form = bank.get_answer_form_for_create(item.ident, [MULTIPLE_CHOICE_ANSWER])
form.set_genus_type(WRONG_ANSWER)
# and "yellow" as the wrong answer
form.add_choice_id('57fe61edcdfc5c3a0bc23b4b')
answer2 = bank.create_answer(form)
print "done creating answers"

done creating answers


Now we can find the `item` `answers` (if you run the above block multiple times, you'll get a varying number of `answer` elements.

In [26]:
item = bank.get_item(item.ident)
print item.get_answers().available()
print [a.object_map for a in item.get_answers()]

5
[{u'displayName': {u'text': u'', u'languageTypeId': '639-2%3AENG%40ISO', u'scriptTypeId': '15924%3ALATN%40ISO', u'formatTypeId': u'TextFormats%3APLAIN%40okapia.net'}, u'description': {u'text': u'', u'languageTypeId': '639-2%3AENG%40ISO', u'scriptTypeId': '15924%3ALATN%40ISO', u'formatTypeId': u'TextFormats%3APLAIN%40okapia.net'}, u'recordTypeIds': [u'answer-record-type%3Amulti-choice%40ODL.MIT.EDU'], 'bankId': u'assessment.Bank%3A57fe54dccdfc5c3a0bc23b47%40ODL.MIT.EDU', u'choiceIds': [u'57fe61edcdfc5c3a0bc23b49'], u'genusTypeId': 'answer-type%3Aright-answer%40ODL.MIT.EDU', u'assignedBankIds': [u'assessment.Bank%3A57fe54dccdfc5c3a0bc23b47%40ODL.MIT.EDU'], 'type': 'Answer', 'id': 'assessment.Answer%3A57fe6605cdfc5c3a0bc23b4c%40ODL.MIT.EDU'}, {u'displayName': {u'text': u'', u'languageTypeId': '639-2%3AENG%40ISO', u'scriptTypeId': '15924%3ALATN%40ISO', u'formatTypeId': u'TextFormats%3APLAIN%40okapia.net'}, u'description': {u'text': u'', u'languageTypeId': '639-2%3AENG%40ISO', u'scriptTyp

Wait ... where are all the wrong `answers`? There should be 2x the number of times you ran the block, `answers` ... and by looking at the `genusTypeId` attributes, it seems like all the `answers` from `get_answers()` are "right-answer" types.

This is because `DLKit` defines the `get_answers()` method to only return the "right" answers, so the "wrong" answers need to be retrieved with a different method, defined by the `item` record extension for wrong answers.

In [28]:
print item.get_wrong_answers().available()
print [a.object_map for a in item.get_wrong_answers()]

4
[{u'displayName': {u'text': u'', u'languageTypeId': '639-2%3AENG%40ISO', u'scriptTypeId': '15924%3ALATN%40ISO', u'formatTypeId': u'TextFormats%3APLAIN%40okapia.net'}, u'description': {u'text': u'', u'languageTypeId': '639-2%3AENG%40ISO', u'scriptTypeId': '15924%3ALATN%40ISO', u'formatTypeId': u'TextFormats%3APLAIN%40okapia.net'}, u'recordTypeIds': [u'answer-record-type%3Amulti-choice%40ODL.MIT.EDU'], 'bankId': u'assessment.Bank%3A57fe54dccdfc5c3a0bc23b47%40ODL.MIT.EDU', u'choiceIds': [u'57fe61edcdfc5c3a0bc23b4b'], u'genusTypeId': 'answer-type%3Awrong-answer%40ODL.MIT.EDU', u'assignedBankIds': [u'assessment.Bank%3A57fe54dccdfc5c3a0bc23b47%40ODL.MIT.EDU'], 'type': 'Answer', 'id': 'assessment.Answer%3A57fe660dcdfc5c3a0bc23b4e%40ODL.MIT.EDU'}, {u'displayName': {u'text': u'', u'languageTypeId': '639-2%3AENG%40ISO', u'scriptTypeId': '15924%3ALATN%40ISO', u'formatTypeId': u'TextFormats%3APLAIN%40okapia.net'}, u'description': {u'text': u'', u'languageTypeId': '639-2%3AENG%40ISO', u'scriptTyp

There they are!

That's a basic overview of `items`. They are very powerful, and to learn more, you can inspect the `records` directory that came along with this tutorial.

##Assessments

Conceptually, `items` are organized into `assessments` -- which are then offered to students. An `assessment` in a classroom might be a homework, a quiz, or an exam ... a tool that evaluates students' knowledge of a topic or outcome.

Let's see what `assessments` exist in our `bank`.

In [30]:
print bank.get_assessments()
print bank.get_assessments().available()

0


If we're starting with a clean slate, we have no `assessments`. Let's get a `form`, and create one. Note that with the current `DLKit` configuration, we need to include at least the `simple-child-sequencing` record for each `assessment`.

In [33]:
from records.registry import ASSESSMENT_RECORD_TYPES
SIMPLE_SEQUENCE_ASSESSMENT = Type(**ASSESSMENT_RECORD_TYPES['simple-child-sequencing'])

form = bank.get_assessment_form_for_create([SIMPLE_SEQUENCE_ASSESSMENT])
form.display_name = 'Homework #1'
form.description = 'Favorites'
assessment = bank.create_assessment(form)
print assessment
print bank.get_assessment_items(assessment.ident).available()

0


Now that we have our `assessment`, we can add `items` to it.

In [34]:
bank.add_item(assessment.ident, item.ident)
print bank.get_assessment_items(assessment.ident).available()

1


This is great for authoring `assessments`, but `DLKit` defines another set of methods to offer the `assessment` in "student mode". This is through the `AssessmentSession` -- but first, we need to create an `assessment_offered`. `assessments_offered` wrap some metadata around the canonical `assessment`, like the `start_time`, `deadline`, `duration`, etc. -- all of these typically are optional. Leaving them out "opens" the `assessment_offered` to students immediately, and it never "closes".

Again, we need a `form` for this, and that method requires the `assessment` ID.

In [35]:
form = bank.get_assessment_offered_form_for_create(assessment.ident, [])
assessment_offered = bank.create_assessment_offered(form)
print assessment_offered

Once we have an `assessment_offered`, we can provide a set of methods for students to take the assessment. Each "taker" generates an `assessment_taken`, which links their taker ID to the `assessment_offered` ID. The `assessment_taken` also links to an `assessment_section`, which maintains a list of questions and responses.

In [37]:
form = bank.get_assessment_taken_form_for_create(assessment_offered.ident, [])
assessment_taken = bank.create_assessment_taken(form)
print assessment_taken
print str(assessment_taken.get_taking_agent_id().identifier)

tutorial_user@school.edu


# Patterns in DLKit

Now that we've gone through the `Assessment` service, you've hopefully learned the basics about catalogs, objects, forms, sessions, and managers. Luckily, these patterns appear across all the `DLKit` services. So you can easily pick up how to use the other services. http://osid.org/ is a good reference, and we've included a simple table below that reflects this `DLKit` build:

````
-------------------------------------------------------------------------------------------------------------
|   Service     |      Catalog     |                            Objects                                     |
-------------------------------------------------------------------------------------------------------------
| Assessment    | Bank             | Item, Assessment, AssessmentOffered, AssessmentTaken, AssessmentPart   |
| Authorization | Vault            | Authorization                                                          |
| Commenting    | Book             | Comment                                                                |
| Grading       | Gradebook        | GradeSystem, GradeEntry, GradebookColumn                               |
| Logging       | Log              | LogEntry                                                               |
| Repository    | Repository       | Asset, AssetContent                                                    |
| Resource      | Bin              | Resource                                                               |
-------------------------------------------------------------------------------------------------------------
````